In [1]:
import pickle as pkl
from utils import *
from utils import *
from tbats import TBATS
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.stats import pearsonr
import statsmodels.api as sm
from pmdarima import auto_arima
from statsmodels.tsa.statespace import exponential_smoothing
from statsmodels.tsa.seasonal import MSTL
# from statsmodels.tsa.api import STLForeca

/Users/julius/repos/AFC/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
data = Data()

In [7]:
tbats_fit = pkl.load(open('fits_pickles/tbats_fit.pkl', 'rb'))
tbats_forecast = pkl.load(open('forecast_pickles/tbats_forecast.pkl', 'rb'))

In [8]:
tbats_fit.summary()

'Use Box-Cox: False\nUse trend: False\nUse damped trend: False\nSeasonal periods: [  7. 365.]\nSeasonal harmonics [3 2]\nARMA errors (p, q): (2, 4)\nSmoothing (Alpha): 0.032816\nSeasonal Parameters (Gamma): [-2.81449412e-06  6.27251353e-06 -9.87982663e-06  7.62772917e-06]\nAR coefficients [-9.87982663e-06  7.62772917e-06]\nMA coefficients [0.2523185  0.19599506 0.05415473 0.07989568]\nSeed vector [1402.99776323  227.02188812   32.03576335  -15.47636037  152.07750758\n   83.99273378   16.43298777 -161.8464263    44.65805918   35.48924498\n   44.90893971    0.            0.            0.            0.\n    0.            0.        ]\n\nAIC 34921.701900'

In [9]:
# Get mean distribution of sales for each product

data.merged_data['total_sales'] = data.merged_data.groupby('day')['sales'].transform('sum')
data.merged_data['percentage_sales'] = (data.merged_data['sales'] / data.merged_data['total_sales'])
mean_percentage_by_product = data.merged_data.groupby('id')['percentage_sales'].mean().reset_index()
print(mean_percentage_by_product)

                              id  percentage_sales
0    FOODS_3_001_TX_3_validation          0.000222
1    FOODS_3_002_TX_3_validation          0.000451
2    FOODS_3_003_TX_3_validation          0.000071
3    FOODS_3_004_TX_3_validation          0.000052
4    FOODS_3_005_TX_3_validation          0.000493
..                           ...               ...
818  FOODS_3_823_TX_3_validation          0.000134
819  FOODS_3_824_TX_3_validation          0.000147
820  FOODS_3_825_TX_3_validation          0.000394
821  FOODS_3_826_TX_3_validation          0.000371
822  FOODS_3_827_TX_3_validation          0.000219

[823 rows x 2 columns]


In [10]:
estimated_sales = (mean_percentage_by_product['percentage_sales'].to_numpy()[:, np.newaxis] * tbats_forecast).round()

#print(sales_test.iloc[:, 1:])

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(data.sales_test.iloc[:, 1:], estimated_sales))

print(f'RMSE: {rmse}')

RMSE: 3.5674739431439835
